### generate the pandas data

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from covid_screening_quantify import quantify_all_fovs, combine_all_quantify, write_conditions

In [ ]:
# test parameter
#image_folder = '/home/haoxu/data/Images'

In [ ]:
#quantify_all_fovs(image_folder)

### combine csv data of all images

In [ ]:
#processed_folder = "/home/haoxu/data/Images_processed"

In [ ]:
#combine_all_quantify(processed_folder)

### generate the pandas data end

### get the conditions

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# write the conditions to csv
processed_folder = "/home/haoxu/data/Images_processed"
quantify_csv_file = os.path.join(processed_folder, 'quantify_all.csv')
condition_xlsx_file = 'test/Plate map test plates 2020-11-25.xlsx'
write_conditions(quantify_csv_file, condition_xlsx_file)

### define cell infection

In [ ]:
import pandas as pd
import os
from imageio import imread
from matplotlib import pyplot as plt
from skimage import segmentation, morphology, exposure
import numpy as np

In [ ]:
processed_folder = "/home/haoxu/data/Images_processed"
df = pd.read_csv(os.path.join(processed_folder, 'quantify_all.csv'))

In [ ]:
# use the max 4 percent in infection channel
plt.hist(df.last4percentmean.to_list(), range=(0, 20000), bins=800)

In [ ]:
# define the threshold value based on the plot above
threshold_value = 4500

In [ ]:
df  = df.assign(**dict.fromkeys(["Infected"], 0))
df.loc[df.last4percentmean > threshold_value, 'Infected'] = 1

### define cell infection end

### verify cell _infection

In [ ]:
well_fovs = list(set(df.well_fov.to_list()))
#well_fovs

In [ ]:
import random
from covid_screening_quantify import show_infections

In [ ]:
# show a random sample to see if the infected cells are correctly picked out.
# if not, fine tune the threshold_value and run
well_fov = random.choice(well_fovs)#'r02c16f04'
print(well_fov)
img = show_infections(df, well_fov, dim_er=True, contour=True, show_er=False)
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(40, 40))
#ax = axes.ravel()
axes.imshow(img/img.max())

### plot plate map for infections

In [ ]:
from covid_screening_quantify import plate_plots

In [ ]:
processed_folder = "/home/haoxu/data/Images_processed"
plate_plots(processed_folder, infection_threshold=4500)

### get analysis result

In [1]:
import os
import numpy as np
import pandas as pd
#from covid_screening_quantify import plate_plots, plate_infection
#infection_data = plate_infection(processed_folder, infection_threshold=3000)
#infection_data

In [2]:
processed_folder = "/home/haoxu/data/Images_processed"
infection_data = os.path.join(processed_folder, 'infection_data.csv')
df_infection = pd.read_csv(infection_data)
df_infection

,well_id,cell_counts,infected_cell_counts,infection_rates,Compound
0,r14c07,5791,1749,30.20,Favipiravir
1,r02c03,5371,1088,20.26,Favipiravir
2,r11c18,6323,1124,17.78,Favipiravir
3,r13c21,2562,1605,62.65,Favipiravir
4,r08c10,4681,2166,46.27,Favipiravir
...,...,...,...,...,...
377,r04c06,5647,1251,22.15,D-Sorbitol
378,r16c08,4270,1699,39.79,D-Sorbitol
379,r04c18,4815,1915,39.77,D-Sorbitol
380,r06c19,4750,1759,37.03,D-Sorbitol


In [ ]:
#df_DMSO = df_infection.loc[df_infection['Compound']=='DMSO']
#df_DMSO = df_DMSO.assign(**dict.fromkeys(["control"], np.nan))
#DMSO_control_columns = ['23', '24']
#df_DMSO["control"] = \
#    df_DMSO.well_id.apply(lambda item: 1 if item[4:] in DMSO_control_columns else 0)
#df_DMSO.loc[(df_DMSO["control"]==1)&(df_DMSO['Compound']=='DMSO')]

### calculate z-prime statistic

In [3]:
df_DMSO = df_infection.loc[df_infection['Compound']=='DMSO']
df_DMSO = df_DMSO.assign(**dict.fromkeys(["DMSO_control"], np.nan))

DMSO_control_columns = ['23', '24']
df_DMSO["DMSO_control"] = df_DMSO.well_id.apply(lambda item: 1 if item[4:] in DMSO_control_columns else 0)

wells_noninfection = df_DMSO[df_DMSO["DMSO_control"]==1]
wells_noninfection_rates = wells_noninfection.infection_rates.to_list()
DMSO_control_rates_stat = {
    'mean': np.mean(wells_noninfection_rates),
    'std': np.std(wells_noninfection_rates)
}

wells_infection = df_DMSO[df_DMSO["DMSO_control"]==0]
wells_infection_rates = wells_infection.infection_rates.to_list()
DMSO_infection_rates_stat = {
    'mean': np.mean(wells_infection_rates),
    'std': np.std(wells_infection_rates)
}

print('DMSO_infection_rates_stat: ', DMSO_infection_rates_stat)
print('DMSO_control_rates_stat: ', DMSO_control_rates_stat)
z_factor = 1 - (
    3 * (DMSO_infection_rates_stat['std'] + DMSO_control_rates_stat['std']) / \
    np.abs(DMSO_infection_rates_stat['mean'] - DMSO_control_rates_stat['mean'])
)
print('z factor: ', z_factor)

DMSO_infection_rates_stat:  {'mean': 36.53902040816325, 'std': 13.467170598843657}
DMSO_control_rates_stat:  {'mean': 7.46, 'std': 10.195188734561677}
z factor:  -1.4411784511244417


In [ ]:
wells_noninfection_rates

In [ ]:
df_DMSO[df_DMSO["DMSO_control"]==2]

In [ ]:
wells_noninfection[wells_noninfection['infection_rates'] > 20]